In [18]:
import mir_eval as me
import os
import numpy as np
import soundfile as sf
import sounddevice as sd
from sklearn.model_selection import KFold

## 1 Load data

In [19]:
'''file0, sr = sf.read("./mir_eval test/mixture.wav" )
file1, _ = sf.read("./mir_eval test/bass.wav")
file2, _ = sf.read("./mir_eval test/drums.wav")
file3, _ = sf.read("./mir_eval test/vocals.wav")
file4, _ = sf.read("./mir_eval test/other.wav")

reference_sources = np.array([file0, file1])
estimated_sources = np.array([file2, file3])
reference_sources'''


'file0, sr = sf.read("./mir_eval test/mixture.wav" )\nfile1, _ = sf.read("./mir_eval test/bass.wav")\nfile2, _ = sf.read("./mir_eval test/drums.wav")\nfile3, _ = sf.read("./mir_eval test/vocals.wav")\nfile4, _ = sf.read("./mir_eval test/other.wav")\n\nreference_sources = np.array([file0, file1])\nestimated_sources = np.array([file2, file3])\nreference_sources'

In [20]:
ref_src_mixes = []
ref_src_vocals = []
est_src_vocals = []
ref_src_acc = []
est_src_acc = []

for filename in os.listdir('../../wav_files/'):
    if 'mix' in filename:
        f, _ = sf.read('../../wav_files/' + filename)
        ref_src_mixes.append(f)
    elif 'vocal_sample' in filename:
        f, _ = sf.read('../../wav_files/' + filename)
        est_src_vocals.append(f)
    elif 'vocals' in filename:
        f, _ = sf.read('../../wav_files/' + filename)
        ref_src_vocals.append(f)
        

In [21]:
for filename in os.listdir('../../wav_files/predicted_acc'):
    f, _ = sf.read('../../wav_files/predicted_acc/' + filename)
    est_src_acc.append(f)
    
for filename in os.listdir('../../wav_files/truth_acc'):
    f, _ = sf.read('../../wav_files/truth_acc/' + filename)
    ref_src_acc.append(f)

In [22]:
ref_src_mixes = np.array(ref_src_mixes)
ref_src_vocals = np.array(ref_src_vocals)
est_src_vocals = np.array(est_src_vocals)
ref_src_acc = np.array(ref_src_acc)
est_src_acc = np.array(est_src_acc)

In [23]:
est_src_vocals.shape

(50, 97536)

In [16]:
ref_src_mixes_clean = []
ref_src_vocals_clean = []
est_src_vocals_clean = []

for i in range(est_src_vocals.shape[0]):
    if np.count_nonzero(ref_src_vocals[i]):
        #print(est_src_vocals[i])
        ref_src_mixes_clean.append(ref_src_mixes[i])
        ref_src_vocals_clean.append(ref_src_vocals[i])
        est_src_vocals_clean.append(est_src_vocals[i])
        
ref_src_mixes_clean = np.array(ref_src_mixes_clean)
ref_src_vocals_clean = np.array(ref_src_vocals_clean)
est_src_vocals_clean = np.array(est_src_vocals_clean)
type(ref_src_mixes_clean)

numpy.ndarray

In [17]:
est_src_vocals_clean.shape

(42, 97536)

## Accompaniment

In [36]:
k = 5

In [32]:
sdr_acc, sir_acc, sar_acc, _ = me.separation.bss_eval_sources(
                                                                    ref_src_acc[0:k], 
                                                                    est_src_acc[0:k], 
                                                                    compute_permutation=True
                                                                    )

In [27]:
sar_acc

array([11.21161343, 12.26606025,  9.82189987])

In [37]:
nsdr_out = nsdr(sdr_acc, ref_src_acc[0:k], est_src_acc[0:k], ref_src_mixes[0:k])
nsdr_out

array([34.17830162, 35.38705424, 32.41358359, 31.32587624,  8.88463086])

## Vocals

In [47]:
sdr_vocal, sir_vocal, sar_vocal, _ = me.separation.bss_eval_sources(
                                                                    ref_src_vocals_clean[0:k], 
                                                                    ref_src_vocals_clean[0:k], 
                                                                    compute_permutation=True
                                                                    )

In [36]:
sar_vocal

array([-12.89981343, -14.23884721, -13.52302672, -12.35486897,
       -14.58652306, -12.83262392, -12.73266704, -13.41414619,
       -14.04276628])

In [49]:
nsdr_out = nsdr(sdr_vocal, ref_src_vocals_clean[0:k], ref_src_vocals_clean[0:k], ref_src_mixes_clean[0:k])
nsdr_out

array([287.21807371, 283.79282929, 301.62199315])

## 2 Metrics

https://craffel.github.io/mir_eval/#module-mir_eval.separation

### 2.1 Evaluate
Computes ```bss_eval_sources``` (for fewer than 3 dimensions) and ```bss_eval_images```.

``` python
mir_eval.separation.evaluate(reference_sources, estimated_sources, **kwargs)
```

#### Returns

	
- **scores:** ```dict```, dictionary of scores, where the key is the metric name (str) and the value is the (float) score achieved.

### 2.2 Blind Source Separation (BSS)
``` python
mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, compute_permutation=True)
```

#### Use

From "WAVE-U-NET: A MULTI-SCALE NEURAL NETWORK FOR END-TO-END AUDIO SOURCE SEPARATION"

_Since the collection of segment-wise vocal SDR values across the dataset is not normally distributed (compare Fig- ure 3 for vocals), the mean and standard deviation are not sufficient to adequately summarise it. As a workaround, **we take the median over segments**, as it is robust against outliers and intuitively describes the minimum performance that is achieved 50% of the time. To describe the spread of the distribution, we use the median absolute deviation (MAD) as a rank-based equivalent to the standard deviation (SD). It is defined as the median of the absolute deviations from the overall median and is easily interpretable, since a value of x means that 50% of values have an absolute difference from the median that is lower than x._

#### Returns


- **sdr:** ```np.ndarray, shape=(nsrc,)```, vector of Signal to Distortion Ratios (SDR)


- **sir:** ```np.ndarray, shape=(nsrc,)```, vector of Source to Interference Ratios (SIR)


- **sar:** ```np.ndarray, shape=(nsrc,)```, vector of Sources to Artifacts Ratios (SAR)


- perm: not used

#### Normalized SDR (NSDR)

$NSDR(S_e, S_r, S_m) = SDR(S_e, S_r) - SDR(S_m, S_r)$

where $S_e$ is the estimated isolated signal, $S_r$ is the reference isolated signal, and $S_m$ is the mixed signal.

In [34]:
def nsdr(sdr, reference_sources, estimated_sources, mix_sources):
    if (False):
        sdr1, _, _, _ = me.separation.bss_eval_sources(reference_sources, estimated_sources, compute_permutation=True)
        sdr2, _, _, _ = me.separation.bss_eval_sources(reference_sources, mix_sources, compute_permutation=True)
    else:
        sdr1 = sdr
        sdr2, _, _, _ = me.separation.bss_eval_sources(reference_sources, mix_sources, compute_permutation=True)
    
    return sdr1 - sdr2

In [55]:
sdr, sir, sar, _ = me.separation.bss_eval_sources(reference_sources, estimated_sources, compute_permutation=True)

In [10]:
sir

array([227.98471633, 227.98471633])

In [11]:
sar

array([4.10238682, 4.10238682])

In [12]:
nsdr_out = nsdr(reference_sources, estimated_sources, mix_sources)
nsdr_out

array([8.18227768, 8.18227768])

### 2.3 Image to spatial distortion

``` python
mir_eval.separation.bss_eval_images(reference_sources, estimated_sources, compute_permutation=True)
```

#### Returns

- **sdr:** ```np.ndarray, shape=(nsrc,)```, vector of Signal to Distortion Ratios (SDR)


- **isr:** ```np.ndarray, shape=(nsrc,)```, vector of source Image to Spatial distortion Ratios (ISR)


- **sir:** ```np.ndarray, shape=(nsrc,)```, vector of Source to Interference Ratios (SIR)


- **sar:** ```np.ndarray, shape=(nsrc,)```, vector of Sources to Artifacts Ratios (SAR)


- perm: not used

In [13]:
sdr, isr, sir, sar, _ = me.separation.bss_eval_images(reference_sources, estimated_sources, compute_permutation=True)

### 2.4 Image to spatial distortion framewise

``` python
mir_eval.separation.bss_eval_images_framewise(reference_sources, estimated_sources, window=1323000, hop=661500, compute_permutation=False)
```

#### Returns

- **sdr:** ```np.ndarray, shape=(nsrc,)```, vector of Signal to Distortion Ratios (SDR)


- **sir:** ```np.ndarray, shape=(nsrc,)```, vector of Source to Interference Ratios (SIR)


- **sar:** ```np.ndarray, shape=(nsrc,)```, vector of Sources to Artifacts Ratios (SAR)


- perm: not used

In [17]:
sdr, isr, sir, sar, _ = me.separation.bss_eval_images_framewise(reference_sources, 
                                                              estimated_sources, 
                                                              window=1323000, 
                                                              hop=661500, 
                                                              compute_permutation=False)

## 3 Evaluate all

In [26]:
_, sirs, sars, _ = me.separation.bss_eval_sources(reference_sources, 
                                                     estimated_sources, 
                                                     compute_permutation=True)

nsdrs = nsdr(reference_sources, estimated_sources, mix_sources)

means = {
    'NSDR Vocal': np.mean(nsdrs),
    'NSDR Instrumental': None,
    'SIR Vocal': np.mean(sirs),
    'SIR Instrumental': None,
    'SAR Vocal': np.mean(sars),
    'SAR Instrumental': None
}
means

{'NSDR Vocal': 8.182277682555021,
 'NSDR Instrumental': None,
 'SIR Vocal': 227.98471632599598,
 'SIR Instrumental': None,
 'SAR Vocal': 4.10238682484913,
 'SAR Instrumental': None}

## 4 Comparison with MIREX results

In [ ]:
mirex_results = {
    'NSDR Vocal': 8.681,
    'NSDR Instrumental': 7.945,
    'SIR Vocal': 15.308,
    'SIR Instrumental': 21.975,
    'SAR Vocal': 11.301,
    'SAR Instrumental': 15.462
    }

#### SAR accompaniment 

In [39]:
sar_mean = np.mean(sar_acc)
sar_mean

10.518666458560723

In [40]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

28.437889310312546

#### SAR vocal

In [85]:
sar_mean = np.mean(sar_vocal)
sar_mean

-12.589121823444003

In [17]:
sar_mean = np.mean(sar_vocal)
sar_mean

-18.059249031308422

In [23]:
sar_mean = np.mean(sar_vocal)
sar_mean

-15.494788654034133

In [31]:
sar_mean = np.mean(sar_vocal)
sar_mean

-14.625165600337708

In [39]:
sar_mean = np.mean(sar_vocal)
sar_mean

-13.402809203017183

#### NSDR Vocal

In [86]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

-12.221848982066094

In [18]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

0.2116603344498517

In [24]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

-11.47433633604592

In [32]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

-12.225159956139196

In [38]:
nsdr_mean = np.mean(nsdr_out)
nsdr_mean

-14.275659510817157